In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,083 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,866 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/multive

In [2]:
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

tar: spark-3.3.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=89578f566057f059b9b72a128f1be94b58e6b4a290f74626868f059fea5ee894
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [4]:
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F


spark = SparkSession \
       .builder \
       .appName("Part-1: Working with SparkSQL") \
       .getOrCreate()

spark

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read the data into a Dataframe.

In [6]:
file_location = "/content/drive/My Drive/data/iot_devices.json"
file_type = "json"

# CSV options
infer_schema = True
first_row_is_header = True
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
iot_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [7]:
iot_df.show(20, False)

+-------------+---------+----+----+-------------+---------+------------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name             |humidity|ip             |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+------------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|8            |868      |US  |USA |United States|1        |meter-gauge-1xbYRYcj    |51      |68.161.225.1   |38.0    |green |-97.0    |Celsius|34  |1458444054093|
|7            |1473     |NO  |NOR |Norway       |2        |sensor-pad-2n2Pea       |70      |213.161.254.1  |62.47   |red   |6.15     |Celsius|11  |1458444054119|
|2            |1556     |IT  |ITA |Italy        |3        |device-mac-36TWSKiT     |44      |88.36.5.1      |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|6            |1080   

Convert the Dataframe into a temporary view called iot.

In [8]:
iot_df.createOrReplaceTempView('iot')

In [9]:
spark.sql("select * from iot").show(5)

+-------------+---------+----+----+-------------+---------+--------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|           ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51| 68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|    88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United States|        4

In [10]:
iot_df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



Count how many devices are there from each country and display the output.

In [24]:
spark.sql("""
SELECT cca3 AS Country_name,
       COUNT(DISTINCT device_id) AS Number_of_devices
FROM iot
GROUP BY cca3
ORDER BY Number_of_devices DESC
""").show()


+------------+-----------------+
|Country_name|Number_of_devices|
+------------+-----------------+
|         USA|            70405|
|         CHN|            14455|
|         JPN|            12100|
|         KOR|            11879|
|         DEU|             7942|
|         GBR|             6486|
|         CAN|             6041|
|         RUS|             5989|
|         FRA|             5305|
|         BRA|             3224|
|         AUS|             3119|
|         ITA|             2915|
|         SWE|             2880|
|         POL|             2744|
|         NLD|             2488|
|         ESP|             2310|
|         TWN|             2128|
|         IND|             1867|
|         CZE|             1507|
|         NOR|             1487|
+------------+-----------------+
only showing top 20 rows



 Display all the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.

In [23]:
spark.sql("""
SELECT Cca2 AS Country_Code,
       cca3 AS Country_name,
       CAST(SUM(c02_level) AS INT) AS CO2_level
FROM iot
WHERE c02_level > 1400
GROUP BY Cca2, cca3
ORDER BY CO2_level DESC
""").show()


+------------+------------+---------+
|Country_Code|Country_name|CO2_level|
+------------+------------+---------+
|          US|         USA| 26242891|
|          CN|         CHN|  5424312|
|          KR|         KOR|  4415118|
|          JP|         JPN|  4399107|
|          DE|         DEU|  2950796|
|          GB|         GBR|  2488574|
|          CA|         CAN|  2343270|
|          RU|         RUS|  2262936|
|          FR|         FRA|  2030583|
|          BR|         BRA|  1284892|
|          AU|         AUS|  1153899|
|          SE|         SWE|  1086146|
|          IT|         ITA|  1070505|
|          PL|         POL|   995721|
|          NL|         NLD|   970297|
|          ES|         ESP|   878143|
|          TW|         TWN|   813253|
|          IN|         IND|   666101|
|          NO|         NOR|   598248|
|          UA|         UKR|   559605|
+------------+------------+---------+
only showing top 20 rows



 Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids (Hint: For high CO2 level, the LCD status will be RED).

In [12]:
spark.sql("select cca3,cn,device_id from iot where lcd = 'red' group by cca3,cn,device_id order by device_id ").show()

+----+-----------------+---------+
|cca3|               cn|device_id|
+----+-----------------+---------+
| NOR|           Norway|        2|
| ITA|            Italy|        3|
| JPN|            Japan|        8|
| USA|    United States|       10|
| ITA|            Italy|       11|
| USA|    United States|       16|
| USA|    United States|       17|
| USA|    United States|       19|
| JPN|            Japan|       22|
| CAN|           Canada|       24|
| KOR|Republic of Korea|       27|
| KOR|Republic of Korea|       28|
| UKR|          Ukraine|       47|
| SWE|           Sweden|       53|
| USA|    United States|       54|
| USA|    United States|       57|
| USA|    United States|       64|
| CZE|   Czech Republic|       66|
| IND|            India|       77|
| KOR|Republic of Korea|       78|
+----+-----------------+---------+
only showing top 20 rows



Find out all devices in countries whose batteries need replacements.

In [14]:
# Assumption - devices with battery level < 3 would need replacement
spark.sql("select cca3, device_id, battery_level from iot where battery_level < 3  order by battery_level ").show()

+----+---------+-------------+
|cca3|device_id|battery_level|
+----+---------+-------------+
| JPN|   106121|            0|
| ZAF|      180|            0|
| DEU|   106203|            0|
| FRA|      345|            0|
| AUS|   106061|            0|
| USA|      185|            0|
| RUS|   106125|            0|
| USA|      179|            0|
| AUS|   106067|            0|
| USA|      195|            0|
| VNM|   106139|            0|
| USA|       17|            0|
| USA|   106076|            0|
| FRA|      196|            0|
| CAN|   106152|            0|
| CAN|       80|            0|
| USA|   106088|            0|
| USA|      209|            0|
| FRA|   106162|            0|
| USA|       98|            0|
+----+---------+-------------+
only showing top 20 rows

